In [ ]:
! pip install -U langchain_community tiktoken langchain-cohere langchainhub chromadb langchain langgraph  tavily-python langchain_groq langchain_google_genai --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 781.3 kB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.1/91.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.2/210.2 kB 12.2 MB/s eta 0:00

In [ ]:
import os
from google.colab import userdata
groq_api_key = userdata.get("GROQ_API_KEY")

In [ ]:
os.environ["LANGCHAIN_TRACING_V2"] = userdata.get("LANGSMITH_API_KEY")
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
langchain_api_key =userdata.get("LANGSMITH_API_KEY")

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    max_tokens=2048,
    timeout=None,
    max_retries=2,
    groq_api_key = groq_api_key
)
print("model retrieved")

model retrieved


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
class domainrouter(BaseModel):
    proj_domain: str = Field(
        description="answer with the domain in which the project lies in , whether in 'webapp' , 'blockchain' , 'mobile' , 'aiml' , 'backend'. Return only the domain name only "
    )

structured_llm_grader = llm.with_structured_output(domainrouter)
system = """You are a professional software developer who has an expertise in the web development ,backend ,  blockchain , aiml and mobile development role ,
    you need to classify whether the project details given by the user lies in which domain , whether it's related to website development , in this case return 'webapp' ,or the project is of backend then return 'backend' or the project is for the blockchain domain and in this case return 'blockchain'
    whether the project is related to AIML then return with the 'aiml' domain , or else if the project contains app development then return with the 'mobile'.
    remember that this decision is very crucial dont give any kind of wrong answer. If the website making or app making is required in aiml or blockchain domain , then categorize them into 'webapp' or 'mobile' only """
route_domain = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "project details : {query} "),
    ]
)
domain_router = route_domain | structured_llm_grader
query = "make me a AAPL stocks dashboard with more than 3 graphs in flask , it should be well detailed "
proj_domain = domain_router.invoke({"query": query})
domain = proj_domain.proj_domain



In [ ]:
print(domain)

webapp


In [ ]:
from re import template
class templaterouter(BaseModel):
    structure_template: str = Field(
        description="decide which template is to be used to make the skeleton codebase of the project , the project domain will be provided. If the domain is 'webapp' you need to classify the template into 'angular', 'NextJS' , 'Astro' , 'REACT_Javascript' , 'REACT_typescript' , 'Simple' , 'Svelte' . If the project domain is Backend development then classify the project into 'Go' , 'Flask' , 'Node' , 'Rust' , 'Laravel' , 'Django' . If the domain is in 'mobile' then classify the project into these templates 'Flutter' , 'react-native' . If the project domain is 'aiml' then classify the project with 'Langchain' . if the domain is 'blockchain' then just simply return 'nil'"
    )

structured_llm_grader = llm.with_structured_output(templaterouter)
system = """You are a professional software developer who has an expertise in the web development , blockchain , aiml and mobile development role ,
    you are extremely specialised in creating the codebases of the complex projects and know which pre existing template to use for various projects , We currently have these templates in the 'webapp' domain which are
     'angular', 'NextJS' , 'Astro' , 'REACT_Javascript' , 'REACT_typescript' , 'SimpleHTML' , 'Svelte' , 'Flask' (Note that if someone wants a website in python you are supposed to return 'Flask') . In the 'backend' domain we have 'Go' , 'Flask' , 'Node' , 'Rust' , 'Laravel' , 'Django' templates . In the 'mobile' domain in which we have to app development we have 'Flutter' , 'react-native' templates . If the domain is AIML , then we have 'langchain' as the domain
     Carefully return me with the project template only on the basis of the domain and the project details. Dont ever return with the domain name , only return the existing templates which are listed in each domain
     Dont return 'webapp' , 'aiml' , 'backend' , 'mobile'. Also if not given which template to use in the user project details then you have to return the most basic template which requires most less files to make """
template_domain = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "project details : {query} , domain : {domain}"),
    ])
template_router = template_domain | structured_llm_grader
template = template_router.invoke({"query": query , "domain": domain})
template_structure = template.structure_template
print(template_structure)


Flask


In [ ]:
from types import new_class
class queryreframer(BaseModel):
    new_query: str = Field(
        description = "You a question re-writer that converts an input question to a better version that is optimized and does not looses any of he context "
    )
structured_llm_grader = llm.with_structured_output(queryreframer)
system = """
You are a professional at reframing the user inputs without loosing the context and the meaning of the query , make changes in the query such that its relevant to the coding project making , make the query such that its relevant to a model which makes projects on the basis of the user queries . Dont make it more complex than it was , and dont add any kind of new functionalities , also dont include user authentication unless stated in the project detail
Return the functionalities as well of the project"""
query_reframing = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "query : {query}"),
    ]
)
query_reframer = query_reframing | structured_llm_grader
new_query_init = query_reframer.invoke({"query": query})
new_query = new_query_init.new_query
print(new_query)

Create a comprehensive AAPL stocks dashboard using Flask, incorporating more than 3 detailed graphs to display various stock metrics and trends, without adding new functionalities or user authentication.


In [ ]:
class templatechecker(BaseModel):
    template_compatibility : str = Field(
        description = "Check whether the given user query can be made with the template given , return 'yes' or 'no' . "
    )
structured_llm_grader = llm.with_structured_output(templatechecker)
system = """You are a professional software developer , you are expert at checking whether the project query given by the user can be made from the template , Return 'yes' if the template is correct and 'no' if the template is incorrect"""
template_checking = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "query : {query} , template : {template}"),
    ]
)
template_checker = template_checking | structured_llm_grader
print(template_checker.invoke({"query": query , "template": template_structure}))

template_compatibility='yes'


In [ ]:
class directorynamer(BaseModel):
    proj_name : str = Field(
        description = "Give the appropriate name for the project"
    )
structured_llm_grader = llm.with_structured_output(directorynamer)
system = """
You are a professional software developer and are extremely professional at giving names to the projects . You have to name the project based on the domain , and the project details . Dont include the domain name in the title , make the title more creative and of single word only """
directory_naming = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "project details : {query} , domain : {domain}"),
    ]
)
directory_namer = directory_naming | structured_llm_grader
project_name = directory_namer.invoke({"query": query , "domain": domain})
root_dir = project_name.proj_name
print(root_dir)

StockSpectra


In [ ]:
print(root_dir)

StockSpectra


In [ ]:
print(template_structure)

Flask


In [ ]:
from code_templates.webapp.simple_html import simple_html_template
from code_templates.webapp.react_js_template import react_js_app_template
from code_templates.webapp.react_ts_template import react_ts_app_template
from code_templates.webapp.flask_app import generate_flask_template
def templatemaker(template_structure , root_dir ):
  if template_structure == "SimpleHTML":
    simple_html_template(root_dir)
  if template_structure == "REACT_Javascript":
    react_js_app_template(root_dir)
  if template_structure == "Flask":
    generate_flask_template(root_dir)
  if template_structure=='REACT_typescript':
    react_ts_app_template(root_dir)


In [ ]:
templatemaker(template_structure , root_dir)

Flask project template created in 'StockSpectra'


In [ ]:
import os

def get_all_files(directory):
    files_list = []

    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            files_list.append(file_path)

    return files_list
all_files = get_all_files(root_dir)
print(all_files)


['StockSpectra/README.md', 'StockSpectra/requirements.txt', 'StockSpectra/app/app.py', 'StockSpectra/app/templates/index.html', 'StockSpectra/app/static/style.css', 'StockSpectra/app/static/main.js']


In [ ]:
class extrafiles(BaseModel):
    extra_files : list = Field(
        description = "Give the extra files required in the project apart from the listed files in python List format"
    )
structured_llm_grader = llm.with_structured_output(extrafiles)
system = """
You are a professional software developer and are extremely professional at listing the appropriate files required to make the project. You have to basically list the files which are required in the projects but are not listed in the file directory .
You also have to recognise the file structure accordingly . You also have the functionalities to add new folder if and only if required . List the extra files required in the python List format . The user input and the list of the existing files are given
Do not ever make duplicate files , and please just try to make the most minimal extra files possible so that its easy for the user. Return an empty list if all the necessary files are already there .
Ignore the license and icon files"""
extra_filing = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "project details : {query} , domain : {domain} , List of existing files : {existing_files}"),
    ]
)
extra_file_lister = extra_filing | structured_llm_grader
project_name = extra_file_lister.invoke({"query": new_query , "domain": domain , "existing_files": all_files})
extra_files = project_name.extra_files
print(extra_files)

['StockSpectra/app/templates/about.html', 'StockSpectra/app/templates/contact.html', 'StockSpectra/app/static/data.csv', 'StockSpectra/app/static/images/aapl_logo.png', 'StockSpectra/app/templates/graphs.html', 'StockSpectra/app/static/js/d3.js', 'StockSpectra/app/static/js/c3.js', 'StockSpectra/app/templates/stock_info.html', 'StockSpectra/app/static/js/plotly.js', 'StockSpectra/app/templates/compare.html']


In [ ]:
#now append extra_files list and all_files into a new list
final_files = all_files + extra_files
print(final_files)

['StockSpectra/README.md', 'StockSpectra/requirements.txt', 'StockSpectra/app/app.py', 'StockSpectra/app/templates/index.html', 'StockSpectra/app/static/style.css', 'StockSpectra/app/static/main.js', 'StockSpectra/app/templates/about.html', 'StockSpectra/app/templates/contact.html', 'StockSpectra/app/static/data.csv', 'StockSpectra/app/static/images/aapl_logo.png', 'StockSpectra/app/templates/graphs.html', 'StockSpectra/app/static/js/d3.js', 'StockSpectra/app/static/js/c3.js', 'StockSpectra/app/templates/stock_info.html', 'StockSpectra/app/static/js/plotly.js', 'StockSpectra/app/templates/compare.html']


In [ ]:
print(final_files[0])

StockSpectra/README.md


In [ ]:
filer_name = final_files[0]

In [ ]:
class CodeGeneration(BaseModel):
    generated_code: dict = Field(
        description="Return the generated code for the corresponding file in JSON format. The keys are the file names and the values are the code content."
    )

# Define the system prompt for code generation
system = """
You are a highly proficient code generator agent with expertise in various programming languages and domains. Your task is to generate the appropriate code for a given file in a project based on the project description, domain, and file name.
You are provided the project details, the domain, and the current file structure.
You must only return the code that should go inside the provided file name and output it in JSON format.
Do not include any explanations, only return the code in JSON format with the file name as the key and the generated code as the value.
"""
code_generation_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Project details: {query}, Domain: {domain}, Existing files: {existing_files}, File to generate code for: {file_name}"),
    ]
)

# Create the structured LLM grader with the output format
structured_llm_grader = llm.with_structured_output(CodeGeneration)
def generate_code(query, domain, all_files, file_name):
    # Invoke the LLM with the structured output
    result = code_generation_prompt | structured_llm_grader
    code_gen = result.invoke({
        "query": query,
        "domain": domain,
        "existing_files": all_files,
        "file_name": file_name
    })

    # Return the generated code in JSON format
    return code_gen.generated_code

In [ ]:
file_name =filer_name
generated_code = generate_code(query, domain, all_files, file_name)

In [ ]:
print(generated_code)


{'Typemonk/style.css': '/* Global Styles */\n\nbody {\n  font-family: Arial, sans-serif;\n  background-color: #f0f0f0;\n  margin: 0;\n  padding: 0;\n}\n\n/* Container Styles */\n\n.container {\n  max-width: 800px;\n  margin: 40px auto;\n  padding: 20px;\n  background-color: #fff;\n  border: 1px solid #ddd;\n  border-radius: 10px;\n  box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);\n}\n\n/* Typing Area Styles */\n\ntyping-area {\n  padding: 20px;\n  font-size: 24px;\n  font-weight: bold;\n  color: #333;\n}\n\ntyping-area input {\n  width: 100%;\n  padding: 10px;\n  font-size: 24px;\n  font-weight: bold;\n  color: #333;\n  border: 1px solid #ccc;\n}\n\ntyping-area input:focus {\n  outline: none;\n  border-color: #aaa;\n}\n\n/* Stats Styles */\n\n.stats {\n  margin-top: 20px;\n  padding: 10px;\n  background-color: #f7f7f7;\n  border: 1px solid #ddd;\n}\n\n.stats span {\n  font-size: 18px;\n  font-weight: bold;\n  color: #666;\n}\n\n/* Button Styles */\n\nbutton {\n  padding: 10px 20px;\n  font-si

In [ ]:
final_files

['Typemonk/style.css',
 'Typemonk/scripts.js',
 'Typemonk/README.md',
 'Typemonk/index.html',
 'Typemonk/manifest.json',
 'Typemonk/words.txt',
 'Typemonk/highscores.json']

In [ ]:
def generate_and_write_code(query, domain, all_files):
    for file_path in final_files:
        file_name = os.path.basename(file_path)

        generated_code = generate_code(query, domain, all_files, file_name)
        code = generated_code.get(file_name, "")

        with open(file_path, 'w') as file:
            file.write(code.replace("\\n", "\n"))

        print(f"Generated and wrote code for {file_name}")

In [ ]:
generate_and_write_code(new_query, domain,final_files)

Generated and wrote code for README.md
Generated and wrote code for requirements.txt
Generated and wrote code for app.py
Generated and wrote code for index.html
Generated and wrote code for style.css
Generated and wrote code for main.js
Generated and wrote code for about.html
Generated and wrote code for contact.html
Generated and wrote code for data.csv


FileNotFoundError: [Errno 2] No such file or directory: 'StockSpectra/app/static/images/aapl_logo.png'

In [ ]:
#now make a downloadable zip file of the folder Typemonk
import shutil
shutil.make_archive(root_dir, 'zip', root_dir)

'/content/StockSpectra.zip'